Colonnes du dataset au départ

'LoanNr_ChkDgt', 'Name', 'City', 'State', 'Zip', 'Bank', 'BankState',
       'NAICS', 'ApprovalDate', 'ApprovalFY', 'Term', 'NoEmp', 'NewExist',
       'CreateJob', 'RetainedJob', 'FranchiseCode', 'UrbanRural', 'RevLineCr',
       'LowDoc', 'ChgOffDate', 'DisbursementDate', 'DisbursementGross',
       'BalanceGross', 'MIS_Status', 'ChgOffPrinGr', 'GrAppv', 'SBA_Appv'
      

Nettoyage de colonnes pour supprimer valeurs nuls et caractères indésirables

In [ ]:
cols_to_convert = ['ApprovalFY', 'DisbursementGross', 'BalanceGross', 'ChgOffPrinGr', 'GrAppv', 'SBA_Appv','NewExist','LowDoc','Bank','BankState']

for col in cols_to_convert:
    df[col] = df[col].astype(str)  # Convertir en string pour nettoyage
    df[col] = df[col].str.replace('[\$,]', '', regex=True)  # Supprimer $ et ,
    df[col] = df[col].str.strip()  # Supprimer les espaces
    df[col] = df[col].replace('', '0')  # Remplacer les valeurs vides par 0
    df[col] = pd.to_numeric(df[col], errors='coerce')  # Convertir en float, mettre NaN si erreur

# Vérifier s'il y a encore des NaN après conversion
print(df[cols_to_convert].isna().sum())  # Compter les valeurs NaN

# Si des NaN sont détectés, les remplacer par 0 ou la moyenne
df[cols_to_convert] = df[cols_to_convert].fillna(0)  # Ou df[cols_to_convert].fillna(df[cols_to_convert].mean())


Convertion de Revline et NewExist pour que le modèle les comprenne

In [ ]:
# Remplacer toutes les valeurs de la variable 'RevLineCr' sauf "Y" et "N" par 0 (unknown)
df.loc[~df['RevLineCr'].isin(['Y', 'N']), 'RevLineCr'] = "0"
# remplacer les valeurs "N" (no) par 1 
df['RevLineCr'] = df['RevLineCr'].str.replace("N", "1")
# remplacer les valeurs "Y" (yes) par 2
df['RevLineCr'] = df['RevLineCr'].str.replace("Y", "2")

# convertir la variable en type int
df['RevLineCr'] = df['RevLineCr'].astype(int)

In [ ]:
df['NewExist'] = df['NewExist'].map({'Existing': 1, 'New': 2, 'Unknown': 0}).astype(int)


Création d'une nouvelle colonne Mis_status_binary/Target

In [ ]:
# Supprimer les espaces supplémentaires dans la colonne 'MIS_Status'
df['MIS_Status'] = df['MIS_Status'].str.strip()

# Remplacer les valeurs manquantes par une valeur par défaut
df['MIS_Status'] = df['MIS_Status'].fillna('UNKNOWN')

# Créer une nouvelle colonne avec les valeurs 0 et 1
df['MIS_Status_binary'] = df['MIS_Status'].apply(lambda x: 1 if x == 'P I F' else (0 if x == 'CHGOFF' else 2))


Création d'une nouvelle colonne Naics_sector pour aider sector

In [ ]:
df['NAICS_sector'] = df['NAICS'].astype(str).str[:2]  # Extrait les 2 premiers chiffres
df['NAICS_sector'] = df['NAICS_sector'].astype(int)